In [1]:
import pandas as pd
import urllib
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import pickle
import preprocessor as p
import numpy as np

In [ ]:
pip install preprocessor

In [ ]:
comments = pd.read_csv('../data/formspring_data_1.csv', sep = '\t', index_col = 0)

In [ ]:
def get_label(post):
    label = 1 if ((post.ans1 == "Yes" and post.ans2 == "Yes") or (post.ans2 == "Yes" and post.ans3 == "Yes") or (post.ans1 == "Yes" and post.ans3 == "Yes")) else 0
    return label

comments['attack'] = comments.apply(get_label, axis=1)

In [ ]:
import string
comments['post'] = comments['post'].apply(lambda x: x.replace('Q:',' ').replace('A:',' ').replace('<br>',' '))
comments['post'] = comments['post'].apply(lambda x: x.replace('Q:',' ').replace('A:',' ').replace('<br>',' '))
comments['comment'] = comments['post'].apply(lambda x: "".join(l for l in x if l not in string.punctuation))
comments = comments[['comment','asker','attack']]
comments.head()

In [ ]:
comments = comments[['comment','asker','attack']]

In [ ]:
comments.head()

In [ ]:
pip install textblob

In [ ]:
swear_words = "../swear_words.txt"
word_list = []
with open(swear_words) as f:
    word_list = f.read().splitlines()

In [ ]:
from textblob import TextBlob

def tokens(post):
    return TextBlob(post).words
    
def get_bad_word_count(post):
    x = tokens(post)
    count = 0
    bad_words = []
    for word in x:
        if word in word_list:
            count+=1
            bad_words.append(word)
    return count, str(bad_words)

comments['bad_word_count'],comments['bad_word_list']= zip(*comments['comment'].map(get_bad_word_count)) 

In [ ]:
def data_analysis(comments):
    total = len(comments)
    bully = len(comments[comments['attack']==1])
    swear = len(comments[comments['bad_word_count']>0])
    anonymous = len(comments[comments['asker']=='None'])
    swear_bully = len(comments[(comments['bad_word_count']>0) & (comments['attack']==1) ])
    anonymous_bully = len(comments[(comments['attack']==1) & (comments['asker']=='None')])
    anonymous_swear = len(comments[(comments['bad_word_count']>0)  & (comments['asker']=='None')])
    anonymous_bully_swear = len(comments[(comments['attack']==1) & (comments['bad_word_count']>0)  & (comments['asker']=='None')])
    
    P_B = float(bully)/total
    P_S = float(swear)/total
    P_A = float(anonymous)/total
    P_B_S = float(swear_bully)/swear
    P_S_B = float(swear_bully)/bully
    P_B_A = float(anonymous_bully)/anonymous
    P_A_B = float(anonymous_bully)/bully
    P_S_A = float(anonymous_swear)/anonymous
    P_A_S = float(anonymous_swear)/total
    P_B_A_S = float(anonymous_bully_swear)/ anonymous_swear
    
    print("P(B): " + str(P_B))
    print("P(S): " + str(P_S))
    print("P(A): " + str(P_A))
    print("P(B|S): " + str(P_B_S))
    print("P(S|B): " + str(P_S_B))
    print("P(B|A): " + str(P_B_A))
    print("P(A|B): " + str(P_A_B))
    print("P(S|A): " + str(P_S_A))
    print("P(B|A & S): " + str(P_B_A_S))
    

In [ ]:
data_analysis(comments)